In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
!pip install llama-index qdrant_client llama-index-llms-ollama torch

In [3]:
!pip install llama-index-embeddings-huggingface

In [4]:
pip install llama-index-vector-stores-postgres llama-index-vector-stores-qdrant

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Download ollama
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread
#Download model
!ollama pull llama3
!pip install ollama
import ollama

#Then everytime you want to chat
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%###############################                                                50.6%#######################################                                                51.1%                          70.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏  58 KB/4.7 GB                  pulling ma

In [6]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True) 

### 1. Setup Embedding Configurations

In [7]:
# Setup LLM using Ollama
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3", request_timeout=120.0)

In [8]:
# Setup Embedding Model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", cache_folder="/kaggle/working/.cache/huggingface/hub/")

# Define global settings embedding model
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

### 2. Load 

#### 2.1 Define Qdrant In-memory store


In [9]:
import qdrant_client
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

client = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    location=":memory:"
    # otherwise set Qdrant instance address with:
    # uri="http://:"
    # set API KEY for Qdrant Cloud
    # api_key="",
)

#### 2.2 Load metadata & extracted features


In [14]:
from llama_index.core import (
    StorageContext,
    Settings,
    SimpleDirectoryReader
)

def load_data(path_to_docs='/kaggle/input/chestdocuments'):
    # Load PDF documents
    required_exts = [".pdf"]
    docs = SimpleDirectoryReader(
        input_dir=path_to_docs,   
        required_exts=required_exts,
        recursive=True,   
        filename_as_id=True 
    ).load_data()
    return docs

In [15]:
docs = load_data()
print(f"Loaded {len(docs)} documents")

Loaded 153 documents


In [17]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(docs)

In [18]:
from llama_index.core import StorageContext

vector_store = QdrantVectorStore(
    client=client, collection_name="chest_disease_collection"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
qdrant_index = VectorStoreIndex(nodes, storage_context=storage_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/llama_index/vector_stores/qdrant/base.py:703: UserWarning: Payload indexes have no effect in the local Qdrant. Please use server Qdrant if you need payload indexes.
  self._client.create_payload_index(


# Querying

In [19]:
def query_chest_documents(query_engine, user_query: str):
    '''Query chest disease documents and return response in Vietnamese'''
    format_prompt = f"""
    Dựa trên tài liệu về bệnh lý ngực, hãy cung cấp thông tin về: "{user_query}"
    
    Yêu cầu:
    - Trả lời bằng tiếng Việt
    - Thông tin phải chính xác và dựa trên tài liệu
    - Nêu rõ nguồn tài liệu nếu có
    - Cung cấp thông tin đầy đủ nhưng súc tích
    """
    response = query_engine.query(format_prompt)
    print(f"Câu hỏi: {user_query}")
    print(f"Trả lời: {response}")


In [20]:
query_engine = qdrant_index.as_query_engine(
    similarity_top_k=5,
    response_mode="tree_summarize"
)

# Example queries
query_chest_documents(query_engine, "Triệu chứng của viêm phổi")
print("-"*100)
query_chest_documents(query_engine, "Phương pháp chẩn đoán ung thư phổi")
print("-"*100)
query_chest_documents(query_engine, "Cách điều trị bệnh lao phổi")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Câu hỏi: Triệu chứng của viêm phổi
Trả lời: Theo tài liệu về bệnh lý ngực, viêm phổi là một tình trạng viêm tại phổi gây ra bởi tác nhân khác nhau. Triệu chứng của viêm phổi bao gồm:

* Đau ngực dữ dội
* Ho ra máu
* Khó thở
* Dấu hiệu sốc
* Tần số thở tăng (>30 lần/phút)
* PaO2/FiO2 dưới 250
* Hình ảnh X-quang cho thấy nhiều mảng thùy phổi
* Lú lẫn, mất định hướng
* Urea máu (BUN > 20 mg/dL)
* Bạch cầu máu < 4000/mm3
* Giảm tiểu cầu (< 100.000/mm3)
* Hạ thân nhiệt (< 36oC)
* Hạ huyết áp cần phải bù dịch tích cực

Nguồn: tài liệu về bệnh lý ngực, trang 29-31 (Phac-do-VP-cong-dong-nguoi-lon_2020.pdf).
----------------------------------------------------------------------------------------------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Câu hỏi: Phương pháp chẩn đoán ung thư phổi
Trả lời: Theo tài liệu, phương pháp chẩn đoán ung thư phổi bao gồm các biện pháp sau:

1. X-quang phổi: Có thể thấy đám mờ trên phim X-quang, giúp chẩn đoán xác định.
2. Chụp cắt lớp vi tính (CT) ngực: Hiển thị bất thường trong phổi, giúp chẩn đoán ung thư phổi.
3. Sinh thiết (Biopsy): Lấy mẫu mô từ phổi và xét nghiệm để chẩn đoán xác định ung thư phổi.

Nguồn tài liệu: Phac-do-VP-cong-dong-nguoi-lon_2020.pdf, trang 3.4.3.
----------------------------------------------------------------------------------------------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Câu hỏi: Cách điều trị bệnh lao phổi
Trả lời: Cách điều trị bệnh lao phổi bao gồm giai đoạn phát hiện và chẩn đoán, sau đó là giai đoạn điều trị bằng thuốc kháng sinh. Giai đoạn điều trị kéo dài khoảng 6 tháng, tuy nhiên có thể ngắn hơn đối với các trường hợp đặc biệt.

Trong quá trình điều trị, người bệnh sẽ được tiêm thuốc kháng sinh vào tĩnh mạch ban đầu, sau đó có thể chuyển đổi sang thuốc uống. Thuốc bao gồm rifampicin, isoniazid, pyrazinamide, và ethambutol.

Người bệnh cần phải được theo dõi và giám sát cẩn thận để phát hiện ra các triệu chứng xấu và kịp thời xử lý. Mục tiêu của chương trình điều trị bệnh lao phổi là giảm triệu chứng, cải thiện chất lượng cuộc sống và gia tăng các hoạt động thể chất và xã hội trong đời sống hàng ngày.

(Nguồn: [59], [60])


In [21]:
query_chest_documents(query_engine,"Liệt kê Tiêu chuẩn nhập khoa Điều trị tích cực của ATS")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Câu hỏi: Liệt kê Tiêu chuẩn nhập khoa Điều trị tích cực của ATS
Trả lời: Theo tài liệu về bệnh lý ngực, Tiêu chuẩn nhập khoa Điều trị tích cực của ATS bao gồm 2 tiêu chuẩn chính và 9 tiêu chuẩn phụ.

Tiêu chuẩn chính bao gồm:

* Suy hô hấp cần phải thông khí cơ học
* Sốc nhiễm khuẩn cần phải dùng thuốc vận mạch

Tiêu chuẩn phụ bao gồm:

* Tần số thở > 30 lần/phút
* PaO2/FiO2 < 250
* T ổn thƣơng nhiều thùy phổi trên phim X-quang
* Lú lẫn, mất định hướng
* Ure máu (BUN > 20 mg/dL)
* Bạch cầu máu < 4000/mm3
* Giảm tiểu cầu (< 100.000/mm3)
* Hạ thân nhiệt (< 36oC)
* Hạ huyết áp cần phải bù dịch tích cực

Nguồn tài liệu: Tài liệu về bệnh lý ngực, trang 27-29.


In [22]:
!pip freeze > requirements.txt